In [1]:
from autogen import AssistantAgent
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

In [2]:
from Util import get_openai_api_key
OPEN_API_KEY= get_openai_api_key()

In [3]:
llm_config={
    "config_list":{
        "model":"gpt-3.5-turbo",
        "api_key":OPEN_API_KEY
    }
}

In [5]:
import pandas as pd
df= pd.read_csv("50_Startups.csv")

In [45]:
converted_dataset= df.to_string(index=False)

In [46]:
converted_dataset

' R&D Spend  Administration  Marketing Spend      State    Profit\n 165349.20       136897.80        471784.10   New York 192261.83\n 162597.70       151377.59        443898.53 California 191792.06\n 153441.51       101145.55        407934.54    Florida 191050.39\n 144372.41       118671.85        383199.62   New York 182901.99\n 142107.34        91391.77        366168.42    Florida 166187.94\n 131876.90        99814.71        362861.36   New York 156991.12\n 134615.46       147198.87        127716.82 California 156122.51\n 130298.13       145530.06        323876.68    Florida 155752.60\n 120542.52       148718.95        311613.29   New York 152211.77\n 123334.88       108679.17        304981.62 California 149759.96\n 101913.08       110594.11        229160.95    Florida 146121.95\n 100671.96        91790.61        249744.55 California 144259.40\n  93863.75       127320.38        249839.44    Florida 141585.52\n  91992.39       135495.07        252664.93 California 134307.35\n 119943.2

In [47]:
from autogen import AssistantAgent

assistant_agent=AssistantAgent(
    name="assistant_agent",
    llm_config=llm_config,
    human_input_mode="NEVER",
    system_message=f"""
    You are a helpful data assistant.
Your task is to analyze a table of data provided to you and display results sorted by profit in descending order.

Instructions:

Read the data carefully (assume it contains a profit column).

Sort the rows in descending order by the profit value.

Return only the sorted table (no explanations, no markdown).

Maintain all original columns in the result.

Here is the dataset:

{converted_dataset}

    """
)

In [49]:
from autogen import ConversableAgent

user_agent= ConversableAgent(
    name="user_agent",
    human_input_mode="NEVER",
    max_consecutive_auto_reply=3,
    code_execution_config={
        "use_docker":False
    }
)

In [51]:
user_agent.initiate_chat(assistant_agent,message="Display the top 5 profit rows")

user_agent (to assistant_agent):

Display the top 5 profit rows

--------------------------------------------------------------------------------
assistant_agent (to user_agent):

165349.20       136897.80        471784.10   New York 192261.83
162597.70       151377.59        443898.53 California 191792.06
153441.51       101145.55        407934.54    Florida 191050.39
144372.41       118671.85        383199.62   New York 182901.99
142107.34        91391.77        366168.42    Florida 166187.94

--------------------------------------------------------------------------------
user_agent (to assistant_agent):



--------------------------------------------------------------------------------
assistant_agent (to user_agent):

165349.20       136897.80        471784.10   New York 192261.83
162597.70       151377.59        443898.53 California 191792.06
153441.51       101145.55        407934.54    Florida 191050.39
144372.41       118671.85        383199.62   New York 182901.99
142107.34  

ChatResult(chat_id=None, chat_history=[{'content': 'Display the top 5 profit rows', 'role': 'assistant', 'name': 'user_agent'}, {'content': '165349.20       136897.80        471784.10   New York 192261.83\n162597.70       151377.59        443898.53 California 191792.06\n153441.51       101145.55        407934.54    Florida 191050.39\n144372.41       118671.85        383199.62   New York 182901.99\n142107.34        91391.77        366168.42    Florida 166187.94', 'role': 'user', 'name': 'assistant_agent'}, {'content': '', 'role': 'assistant', 'name': 'user_agent'}, {'content': '165349.20       136897.80        471784.10   New York 192261.83\n162597.70       151377.59        443898.53 California 191792.06\n153441.51       101145.55        407934.54    Florida 191050.39\n144372.41       118671.85        383199.62   New York 182901.99\n142107.34        91391.77        366168.42    Florida 166187.94', 'role': 'user', 'name': 'assistant_agent'}, {'content': '', 'role': 'assistant', 'name': '

In [32]:
with open("50_Startups.csv","r") as file:
    csv_format=file.read()

In [33]:
class FileReadingUserProxy(RetrieveUserProxyAgent):
    def retrieve_user_input(self,prompt=None):
        return "Here is the dataset:\n" +csv_format

In [28]:
profit_csv_data= df.to_csv(index=False)
profit_csv_data

'R&D Spend,Administration,Marketing Spend,State,Profit\n165349.2,136897.8,471784.1,New York,192261.83\n162597.7,151377.59,443898.53,California,191792.06\n153441.51,101145.55,407934.54,Florida,191050.39\n144372.41,118671.85,383199.62,New York,182901.99\n142107.34,91391.77,366168.42,Florida,166187.94\n131876.9,99814.71,362861.36,New York,156991.12\n134615.46,147198.87,127716.82,California,156122.51\n130298.13,145530.06,323876.68,Florida,155752.6\n120542.52,148718.95,311613.29,New York,152211.77\n123334.88,108679.17,304981.62,California,149759.96\n101913.08,110594.11,229160.95,Florida,146121.95\n100671.96,91790.61,249744.55,California,144259.4\n93863.75,127320.38,249839.44,Florida,141585.52\n91992.39,135495.07,252664.93,California,134307.35\n119943.24,156547.42,256512.92,Florida,132602.65\n114523.61,122616.84,261776.23,New York,129917.04\n78013.11,121597.55,264346.06,California,126992.93\n94657.16,145077.58,282574.31,New York,125370.37\n91749.16,114175.79,294919.57,Florida,124266.9\n86419

In [34]:
system_assistant=AssistantAgent(
    name="system_assistant",
    human_input_mode="NEVER",
    llm_config=llm_config,
    code_execution_config={
        "use_docker":False
    },
    system_message="""You are a data analysis assistant. Given a dataset containing a 'Profit' column, your task is to sort the dataset in descending order of profit and display the top 10 rows with the highest profit values as output. Ensure the output includes the entire row for each of the top 10 entries."""
)

In [30]:
from autogen.retrieve_utils import LocalFileRetriever

ImportError: cannot import name 'LocalFileRetriever' from 'autogen.retrieve_utils' (/opt/anaconda3/lib/python3.12/site-packages/autogen/retrieve_utils.py)

In [31]:
import autogen.retrieve_utils as ru
print(dir(ru))

['API', 'Any', 'BeautifulSoup', 'Callable', 'HAS_UNSTRUCTURED', 'Optional', 'QueryResult', 'RAG_MINIMUM_MESSAGE_LENGTH', 'TEXT_FORMATS', 'UNSTRUCTURED_FORMATS', 'Union', 'VALID_CHUNK_MODES', '__builtins__', '__cached__', '__doc__', '__file__', '__loader__', '__name__', '__package__', '__spec__', '_generate_file_name_from_url', 'chromadb', 'count_token', 'create_vector_db_from_dir', 'ef', 'extract_text_from_pdf', 'get_file_from_url', 'get_files_from_dir', 'glob', 'hashlib', 'is_url', 'logger', 'logging', 'markdownify', 'optional_import_block', 'os', 'parse_html_to_markdown', 'pypdf', 'query_vector_db', 're', 'requests', 'require_optional_import', 'result', 'split_files_to_chunks', 'split_text_to_chunks', 'urlparse']


In [38]:
from autogen.agentchat.contrib.retrieve_user_proxy_agent import RetrieveUserProxyAgent

raguserproxy=FileReadingUserProxy(
    name="raguserproxy",
    is_termination_msg=lambda x: "TERMINATE" in x,
    human_input_mode="NEVER",
    max_consecutive_auto_reply=2,
    code_execution_config={
        "use_docker":False
    },
    retrieve_config={
        "task":"Sort the dataset by 'Profit' in descending order and return it as CSV."
        "docs_path":["50_Startups.csv"]
    }
)

SyntaxError: invalid syntax (2701888923.py, line 13)

In [36]:
from autogen import GroupChat,GroupChatManager
group_chat= GroupChat(agents=[system_assistant,raguserproxy],messages=[],max_round=5)
manager=GroupChatManager(groupchat=group_chat,llm_config=llm_config)

In [37]:
user_prompt="Can you display the top 10 rows sorted by Profit in descending order?"
results=raguserproxy.initiate_chat(manager,message=user_prompt)
results

raguserproxy (to chat_manager):

Can you display the top 10 rows sorted by Profit in descending order?

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Sure! Please provide me with the dataset containing the 'Profit' column so that I can analyze and sort it accordingly.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Once you provide the dataset, I will be able to assist you further.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Please provide the dataset with the 'Profit' column so that I can display the top 10 rows sorted by profit in descending order.

--------------------------------------------------------------------------------


GroupChat is underpopulated with 2 agents. Consider setting speaker_selection_method to 'round_robin' or allow_repeat_speaker to False, or use direct communication, unless repeated speaker is desired.



Next speaker: system_assistant

system_assistant (to chat_manager):

Please provide the dataset with the 'Profit' column so that I can analyze it and show you the top 10 rows sorted by profit in descending order.

--------------------------------------------------------------------------------

>>>>>>>> TERMINATING RUN (4a001f93-b3fd-4b63-b140-525252a9bfd4): Maximum rounds (5) reached


ChatResult(chat_id=None, chat_history=[{'content': 'Can you display the top 10 rows sorted by Profit in descending order?', 'role': 'assistant', 'name': 'raguserproxy'}, {'content': "Sure! Please provide me with the dataset containing the 'Profit' column so that I can analyze and sort it accordingly.", 'name': 'system_assistant', 'role': 'user'}, {'content': 'Once you provide the dataset, I will be able to assist you further.', 'name': 'system_assistant', 'role': 'user'}, {'content': "Please provide the dataset with the 'Profit' column so that I can display the top 10 rows sorted by profit in descending order.", 'name': 'system_assistant', 'role': 'user'}, {'content': "Please provide the dataset with the 'Profit' column so that I can analyze it and show you the top 10 rows sorted by profit in descending order.", 'name': 'system_assistant', 'role': 'user'}], summary="Please provide the dataset with the 'Profit' column so that I can analyze it and show you the top 10 rows sorted by profi